In [1]:
try:
  %tensorflow_version 2.x
except Exception:
  pass

import tensorflow as tf
print(tf.__version__)
import IPython.display as display
import matplotlib.pyplot as plt
import random
import time
import pathlib
from sklearn.model_selection import train_test_split

TensorFlow 2.x selected.
2.0.0


In [32]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [14]:
data_root = pathlib.Path('/content/drive/My Drive/Smash_Bros_Master/_stacks_mini/')
label_names = sorted(item.name for item in data_root.glob('*/') if item.is_dir())
print(label_names)

['bg', 'falcon', 'fox', 'pikachu']


In [0]:
IMG_SIZE = 32
# AUTOTUNE = tf.data.experimental.AUTOTUNE
BATCH_SIZE = 32
IMG_SIZE = 32
# SHUFFLE_SIZE = 1000

In [28]:
from tensorflow.keras import layers, models
model = models.Sequential()
model.add(layers.Conv2D(8, (3, 3), activation='relu', 
                        input_shape=(IMG_SIZE, IMG_SIZE, 3)))
model.add(layers.MaxPooling2D())
model.add(layers.Conv2D(16, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D())
model.add(layers.Flatten())
model.add(layers.Dropout(rate=0.1))
model.add(layers.Dense(4, activation='softmax'))
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_8 (Conv2D)            (None, 30, 30, 8)         224       
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 15, 15, 8)         0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 13, 13, 16)        1168      
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 6, 6, 16)          0         
_________________________________________________________________
flatten_4 (Flatten)          (None, 576)               0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 576)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 4)                

In [29]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255,
                                   zoom_range=0.9,
                                   width_shift_range=0.1,
                                   height_shift_range=0.1)

# train_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        data_root,
        target_size=(IMG_SIZE, IMG_SIZE),
        batch_size=32,
        class_mode='sparse')

for data_batch, labels_batch in train_generator:
    print('data batch shape:', data_batch.shape)
    print('labels batch shape:', labels_batch.shape)
    break
    
history = model.fit_generator(
      train_generator,
      steps_per_epoch=230,
      epochs=5)

Found 7357 images belonging to 4 classes.
data batch shape: (32, 32, 32, 3)
labels batch shape: (32,)
Epoch 1/5
230/230 [==============================] - 14s 62ms/step - loss: 0.4612 - accuracy: 0.8483
Epoch 2/5
230/230 [==============================] - 14s 62ms/step - loss: 0.1258 - accuracy: 0.9602
Epoch 3/5
230/230 [==============================] - 15s 63ms/step - loss: 0.0913 - accuracy: 0.9731
Epoch 4/5
230/230 [==============================] - 14s 62ms/step - loss: 0.0747 - accuracy: 0.9750
Epoch 5/5
230/230 [==============================] - 14s 63ms/step - loss: 0.0664 - accuracy: 0.9796


In [0]:
from google.colab import files
model.save("/content/stack_model_9.h5")
files.download("/content/stack_model_9.h5")

In [0]:
class DataSetGenerator():
    def __init__(self, IMG_HEIGHT, IMG_WIDTH):
        self.IMG_HEIGHT = IMG_HEIGHT
        self.IMG_WIDTH = IMG_WIDTH

    @tf.function
    def load_and_preprocess_image(self, path):
        img = tf.io.read_file(path)
        img = tf.image.decode_jpeg(img, channels=3)
        img = tf.image.resize(img, [self.IMG_HEIGHT, self.IMG_WIDTH])
        img /= 255.0  # normalize to [0,1] range
        return img

    def prepare_for_training(self, ds, cache=True):
        # This is a small dataset, only load it once, and keep it in memory.
        # use `.cache(filename)` to cache preprocessing work for datasets that don't
        # fit in memory.
        if cache:
            if isinstance(cache, str):
                ds = ds.cache(cache)
        else:
          ds = ds.cache()
        
        ds = ds.shuffle(buffer_size=self.SHUFFLE_SIZE)

        ds = ds.batch(self.BATCH_SIZE)

        # `prefetch` lets the dataset fetch batches in the background while the model
        # is training.
        ds = ds.prefetch(buffer_size=self.AUTOTUNE)

        return ds

    def labeled_dataset(self, image_paths, labels):
        # a dataset that returns image paths
        path_ds = tf.data.Dataset.from_tensor_slices(image_paths)
        # for n, img_path in enumerate(path_ds.take(4)):
        #     print(n, img_path)

        # a dataset that returns images (loaded off disk, decoded, and preprocessed)
        image_ds = path_ds.map(self.load_and_preprocess_image, num_parallel_calls=self.AUTOTUNE)
        # for n, image in enumerate(image_ds.take(4)):
        #     print(n, image.shape)

        # a dataset that returns labels
        label_ds = tf.data.Dataset.from_tensor_slices(tf.cast(labels, tf.int64))
        # for label in label_ds.take(4):
        #     print(self.label_names[label.numpy()])

        # a dataset that returns images and labels
        image_label_ds = tf.data.Dataset.zip((image_ds, label_ds))
        # for img, label in image_label_ds.take(2):
        #     print(img.shape, self.label_names[label.numpy()])

        return image_label_ds

    def get_final_dataset(self, data_root, SHUFFLE_SIZE, BATCH_SIZE):
        self.SHUFFLE_SIZE =SHUFFLE_SIZE
        self.BATCH_SIZE = BATCH_SIZE
        self.AUTOTUNE = tf.data.experimental.AUTOTUNE

        # a dataset that returns image paths
        data_root = pathlib.Path(data_root)
        all_image_paths = list(data_root.glob('*/*'))
        all_image_paths = [str(path) for path in all_image_paths if str(path).lower().endswith("png")]
        random.shuffle(all_image_paths)
        print('There are %d images in this dataset.' % len(all_image_paths))

        self.label_names = sorted(item.name for item in data_root.glob('*/') if item.is_dir())
        print('All categories in the dataset:', self.label_names)
        label_to_index = dict((name, index) for index, name in enumerate(self.label_names))
        all_labels = [label_to_index[pathlib.Path(path).parent.name]
                        for path in all_image_paths]

        # separate dataset
        train_paths, test_paths, train_labels, test_labels = train_test_split(all_image_paths, all_labels)

        train_ds = self.labeled_dataset(train_paths, train_labels)
        train_ds = self.prepare_for_training(train_ds)

        test_ds = self.labeled_dataset(test_paths, test_labels)
        test_ds = self.prepare_for_training(test_ds)

        return train_ds, test_ds


In [15]:
IMG_SIZE = 32
BATCH_SIZE = 32
SHUFFLE_SIZE = 20000
AUTOTUNE = tf.data.experimental.AUTOTUNE
data_root = pathlib.Path('/content/drive/My Drive/Smash_Bros_Master/_stacks_mini/')
dsg = DataSetGenerator(IMG_SIZE,IMG_SIZE)
train_ds, test_ds = dsg.get_final_dataset(data_root, SHUFFLE_SIZE, BATCH_SIZE)

There are 7359 images in this dataset.
All categories in the dataset: ['bg', 'falcon', 'fox', 'pikachu']


In [16]:
from tensorflow.keras import layers, models
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', 
                        input_shape=(IMG_SIZE, IMG_SIZE, 3)))
model.add(layers.MaxPooling2D())
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D())
model.add(layers.Flatten())
model.add(layers.Dropout(rate=0.2))
model.add(layers.Dense(4, activation='softmax'))
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 30, 30, 32)        896       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 13, 13, 64)        18496     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 6, 6, 64)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 2304)              0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 2304)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 4)                

In [0]:
es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=2, mode='auto')
history = model.fit(train_ds, validation_data=test_ds, epochs=10, callbacks=[es_callback])